In [2]:
from dask_cassandra_loader import Loader

dask_cassandra_loader = Loader()


In [3]:
keyspace = 'tutorial'
cluster = ['127.0.0.1']

In [4]:
dask_cassandra_loader.connect_to_cassandra(cluster,
                                           keyspace,
                                           username='cassandra',
                                           password='cassandra')

In [5]:
dask_cassandra_loader.connect_to_local_dask()

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at: inproc://192.168.16.112/4611/1
distributed.scheduler - INFO -   dashboard at:            localhost:8787
distributed.worker - INFO -       Start worker at: inproc://192.168.16.112/4611/3
distributed.worker - INFO -          Listening to:       inproc192.168.16.112
distributed.worker - INFO -          dashboard at:       192.168.16.112:41733
distributed.worker - INFO - Waiting to connect to: inproc://192.168.16.112/4611/1
distributed.worker - INFO - -------------------------------------------------
distributed.worker - INFO -               Threads:                         12
distributed.worker - INFO -                Memory:                   26.67 GB
distributed.worker - INFO -       Local Directory: /home/dev/Practice/dask/myjupyter/course/dask-worker-space/worker-dwssikx2
distributed.worker - INFO - -------------------------------------------------
distributed.scheduler - INFO - Register wor

In [6]:
table = dask_cassandra_loader.load_cassandra_table('tab1',
                                         ['id', 'year', 'month', 'day'],
                                         [('day', 'equal', [8])],
                                         [('id', [18]), ('year', [2018]),
                                          ('month', [11])],
                                         force=False)

In [7]:
if table is None:
    raise AssertionError("Table is not supposed to be None!!!")

if table.data is None:
    raise AssertionError("Table.data is not supposed to be None!!!")

In [8]:
local_table = table.data.compute()

In [9]:
print(local_table.head())

   id  year  month  day
0  18  2018     11    8


In [15]:
table2 = dask_cassandra_loader.load_cassandra_table('tab1',
                                         ['id', 'year', 'month', 'day'],[],[],
                                         force=True)

In [16]:
local_table2 = table2.data.compute()

In [17]:
local_table2.head()

,id,year,month,day
0,18,2018,12,8
0,18,2018,11,8
